In [1]:
from PIL import Image
import os, glob, numpy as np #이미지 프로세싱
from sklearn.model_selection import train_test_split #train set과 test set으로 분류 

# 데이터셋 저장한 디렉토리
base_dir = './KWGuideImg/dataset/train'
categories=os.listdir(base_dir) #["기념","화도","옥의","비마","새빛","참빛","한울","연구"]
nb_classes=len(categories) #class 개수 설정

In [2]:
from keras.preprocessing.image import ImageDataGenerator

#ImageDataGenerator 클래스 사용하여 이미지 데이터 증식

datagen = ImageDataGenerator(
    rotation_range=40, #이미지 회전 범위
    width_shift_range=0.2, #그림 수평 or 수직으로 랜덤하게 평행이동시키는 범위
    height_shift_range=0.2,
    rescale=1./255, #1/255로 스케일링하여 0~1범위로 변환하여 효과적으로 학습
    shear_range=0.2, #임의 전단 변환
    zoom_range=0.2, #임의 확대/축소 범위
    horizontal_flip=False, #이미지를 50%확률로 수평으로 뒤집음(뒤집으면 자연스럽지 않으므로, False) 
    fill_mode='nearest') #이미지를 회전, 이동하거나 축소할 때 생기는 공간 채움

Using TensorFlow backend.


In [3]:
# 훈련용 80주념기념관 사진 디렉터리
train_ginyeom_dir = './KWGuideImg/dataset/train/ginyeom'

# 훈련용 화도관 사진 디렉터리
train_hwado_dir = './KWGuideImg/dataset/train/hwado'

# 훈련용 새빛관 사진 디렉터리
train_saebit_dir = './KWGuideImg/dataset/train/saebit'

# 훈련용 비마관 사진 디렉터리
train_bima_dir = './KWGuideImg/dataset/train/bima'

# 훈련용 옥의관 사진 디렉터리
train_oakui_dir = './KWGuideImg/dataset/train/oakeui'

# 훈련용 한천재 사진 디렉터리
train_hancheon_dir = './KWGuideImg/dataset/train/hanchoen'

# 훈련용 복지관 사진 디렉터리
train_bokji_dir = './KWGuideImg/dataset/train/bok'

# 훈련용 연구관 사진 디렉터리
train_yeonku_dir = './KWGuideImg/dataset/train/yeon'

# 훈련용 참빛관 사진 디렉터리
train_chambit_dir = './KWGuideImg/dataset/train/chambit'

# 훈련용 한울관 사진 디렉터리
train_hanoul_dir = './KWGuideImg/dataset/train/hanoul'

In [4]:
# 이미지 전처리 유틸리티 모듈
from keras.preprocessing import image

fnames1 = sorted([os.path.join(train_ginyeom_dir, fname) for fname in os.listdir(train_ginyeom_dir)])
fnames2 = sorted([os.path.join(train_hwado_dir, fname) for fname in os.listdir(train_hwado_dir)])
fnames3 = sorted([os.path.join(train_saebit_dir, fname) for fname in os.listdir(train_saebit_dir)])
fnames4 = sorted([os.path.join(train_bima_dir, fname) for fname in os.listdir(train_bima_dir)])
fnames5 = sorted([os.path.join(train_oakui_dir, fname) for fname in os.listdir(train_oakui_dir)])
fnames6 = sorted([os.path.join(train_chambit_dir, fname) for fname in os.listdir(train_chambit_dir)])
fnames7 = sorted([os.path.join(train_hanoul_dir, fname) for fname in os.listdir(train_hanoul_dir)])

#이미지 데이터 증식 (한번만 수행)

for img_path in fnames1:

    img = image.load_img(img_path, target_size=(150, 150)) # 이미지 읽고 크기 변경

    x = image.img_to_array(img) # (150, 150, 3) 크기의 넘파이 배열

    x = x.reshape((1,) + x.shape) # (1, 150, 150, 3) 크기의 넘파이 배열

    # flow() 메서드로 랜덤하게 변환된 이미지를 배치 단위로 생성하여 폴더에 저장
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                             save_to_dir='./KWGuideImg/dataset/train/gi',save_format='jpg'):
        i += 1
        if i > 10:
            break #이미지 10장 생성

for img_path in fnames2:

    img = image.load_img(img_path, target_size=(150, 150)) # 이미지 읽고 크기 변경

    x = image.img_to_array(img) # (150, 150, 3) 크기의 넘파이 배열

    x = x.reshape((1,) + x.shape) # (1, 150, 150, 3) 크기의 넘파이 배열

    # flow() 메서드로 랜덤하게 변환된 이미지를 배치 단위로 생성하여 폴더에 저장
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                             save_to_dir='./KWGuideImg/dataset/train/hwan',save_format='jpg'):
        i += 1
        if i > 10:
            break #이미지 10장 생성
for img_path in fnames3:

    img = image.load_img(img_path, target_size=(150, 150)) # 이미지 읽고 크기 변경

    x = image.img_to_array(img) # (150, 150, 3) 크기의 넘파이 배열

    x = x.reshape((1,) + x.shape) # (1, 150, 150, 3) 크기의 넘파이 배열

    # flow() 메서드로 랜덤하게 변환된 이미지를 배치 단위로 생성하여 폴더에 저장
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                             save_to_dir='./KWGuideImg/dataset/train/se',save_format='jpg'):
        i += 1
        if i > 10:
            break #이미지 10장 생성
for img_path in fnames4:

    img = image.load_img(img_path, target_size=(150, 150)) # 이미지 읽고 크기 변경

    x = image.img_to_array(img) # (150, 150, 3) 크기의 넘파이 배열

    x = x.reshape((1,) + x.shape) # (1, 150, 150, 3) 크기의 넘파이 배열

    # flow() 메서드로 랜덤하게 변환된 이미지를 배치 단위로 생성하여 폴더에 저장
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                             save_to_dir='./KWGuideImg/dataset/train/bi',save_format='jpg'):
        i += 1
        if i > 10:
            break #이미지 10장 생성
for img_path in fnames5:

    img = image.load_img(img_path, target_size=(150, 150)) # 이미지 읽고 크기 변경

    x = image.img_to_array(img) # (150, 150, 3) 크기의 넘파이 배열

    x = x.reshape((1,) + x.shape) # (1, 150, 150, 3) 크기의 넘파이 배열

    # flow() 메서드로 랜덤하게 변환된 이미지를 배치 단위로 생성하여 폴더에 저장
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                             save_to_dir='./KWGuideImg/dataset/train/oak',save_format='jpg'):
        i += 1
        if i > 10:
            break #이미지 10장 생성
for img_path in fnames6:

    img = image.load_img(img_path, target_size=(150, 150)) # 이미지 읽고 크기 변경

    x = image.img_to_array(img) # (150, 150, 3) 크기의 넘파이 배열

    x = x.reshape((1,) + x.shape) # (1, 150, 150, 3) 크기의 넘파이 배열

    # flow() 메서드로 랜덤하게 변환된 이미지를 배치 단위로 생성하여 폴더에 저장
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                             save_to_dir='./KWGuideImg/dataset/train/cham',save_format='jpg'):
        i += 1
        if i > 10:
            break #이미지 10장 생성

for img_path in fnames7:

    img = image.load_img(img_path, target_size=(150, 150)) # 이미지 읽고 크기 변경

    x = image.img_to_array(img) # (150, 150, 3) 크기의 넘파이 배열

    x = x.reshape((1,) + x.shape) # (1, 150, 150, 3) 크기의 넘파이 배열

    # flow() 메서드로 랜덤하게 변환된 이미지를 배치 단위로 생성하여 폴더에 저장
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                             save_to_dir='./KWGuideImg/dataset/train/hanoul',save_format='jpg'):
        i += 1
        if i > 10:
            break #이미지 10장 생성            



In [5]:
image_w = 64
image_h = 64 #크기지정

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):  #각 건물마다
    
    label = [0 for i in range(nb_classes)] #건물 카테고리 개수
    label[idx] = 1

    image_ = base_dir + "/" + cat
    files = glob.glob(image_+"/*.jpg")
    print(cat, " File Size : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB") #RGB로 변환 
        img = img.resize((image_w, image_h)) #크기 (64,64)로 변환
        data = np.asarray(img) #numpy.ndarray()형식으로 변환

        X.append(data) #train image
        y.append(label) #train label 

        if i % 700 == 0:
            print(cat, " : ", f)

X = np.array(X)
y = np.array(y)
#1 0 0 0 0 0 0 이면 기념관
#0 1 0 0 0 0 0이면 화도관


X_train, X_test, y_train, y_test = train_test_split(X, y) #트레인이미지와 테스트이미지 나눔
xy = (X_train, X_test, y_train, y_test)
np.save('dataset_numpy.npy', xy) #("C:/KWGuideImg/dataset/test", xy) 

print("ok", len(y))

bima  File Size :  424
bima  :  ./KWGuideImg/dataset/train/bima\image_024.jpg
bok  File Size :  7
bok  :  ./KWGuideImg/dataset/train/bok\image_014.jpg
chambit  File Size :  837
chambit  :  ./KWGuideImg/dataset/train/chambit\image_029.jpg
chambit  :  ./KWGuideImg/dataset/train/chambit\_0_8364.jpg
ginyeom  File Size :  2341
ginyeom  :  ./KWGuideImg/dataset/train/ginyeom\image_001.jpg
ginyeom  :  ./KWGuideImg/dataset/train/ginyeom\_0_3764.jpg
ginyeom  :  ./KWGuideImg/dataset/train/ginyeom\_0_6489.jpg
ginyeom  :  ./KWGuideImg/dataset/train/ginyeom\_0_9112.jpg
hanchoen  File Size :  1
hanchoen  :  ./KWGuideImg/dataset/train/hanchoen\image_035.jpg
hanoul  File Size :  423
hanoul  :  ./KWGuideImg/dataset/train/hanoul\image_031.jpg
hwado  File Size :  1222
hwado  :  ./KWGuideImg/dataset/train/hwado\image_011.jpg
hwado  :  ./KWGuideImg/dataset/train/hwado\_0_6216.jpg
oakeui  File Size :  796
oakeui  :  ./KWGuideImg/dataset/train/oakeui\image_025.jpg
oakeui  :  ./KWGuideImg/dataset/train/oakeui\

In [6]:
#numpy 데이터 불러옴
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K
import tensorflow as tf

config =  tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

X_train, X_test, y_train, y_test = np.load('dataset_numpy.npy',allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])
print(y_train.shape)

(5167, 64, 64, 3)
5167
(5167, 10)


In [7]:
categories=os.listdir(base_dir)
nb_classes = len(categories)

#train/ test분류
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255
print ("ok")

ok


In [8]:
from keras.models import load_model

with K.tf_ops.device('/device:CPU:0'):
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_dir = './model'
    
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    model_path = model_dir + '/multi_img_classification.model'
    checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=6)
    
    print("ok")

ok


In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16384)            

In [11]:
from keras.models import load_model

history = model.fit(X_train, y_train, batch_size=32, epochs=30)

print("정확도 : %.4f" % (model.evaluate(X_test, y_test)[1]))

model.save('Building_cnn.h5')

Epoch 1/30
5167/5167 [==============================] - 24s 5ms/step - loss: 1.7858 - accuracy: 0.3331
Epoch 2/30
5167/5167 [==============================] - 29s 6ms/step - loss: 1.3517 - accuracy: 0.3753
Epoch 3/30
5167/5167 [==============================] - 27s 5ms/step - loss: 0.9227 - accuracy: 0.6623
Epoch 4/30
5167/5167 [==============================] - 27s 5ms/step - loss: 0.5859 - accuracy: 0.8028
Epoch 5/30
5167/5167 [==============================] - 28s 5ms/step - loss: 0.3923 - accuracy: 0.8698
Epoch 6/30
5167/5167 [==============================] - 32s 6ms/step - loss: 0.2765 - accuracy: 0.9087
Epoch 7/30
5167/5167 [==============================] - 27s 5ms/step - loss: 0.2192 - accuracy: 0.9313
Epoch 8/30
5167/5167 [==============================] - 27s 5ms/step - loss: 0.1751 - accuracy: 0.9462
Epoch 9/30
5167/5167 [==============================] - 27s 5ms/step - loss: 0.1576 - accuracy: 0.9491
Epoch 10/30
5167/5167 [==============================] - 27s 5ms/step - l

In [12]:
msg='./KWGuideImg/dataset/test/test_img2.jpg'
print(msg)
img = Image.open(msg)
img = img.convert("RGB")
image_w = 64
image_h = 64 #크기지정
img = img.resize((image_w, image_h)) #사이즈 재조정
data = np.asarray(img)
X=np.array(data)

X = X.astype("float") / 256
X = X.reshape(-1, 64, 64,3)

#훈련된 모델을 사용하여 이미지 예측
model=load_model('Building_cnn.h5')
predictions = model.predict(X)  #X이미지 레이블 예측
predictions[0]
result = np.argmax(predictions[0])    # 예측 값중 가장 높은 신뢰도를 가진 레이블
base_dir = './KWGuideImg/dataset/train'
categories=os.listdir(base_dir) #["기념","화도","옥의","비마","새빛","참빛","한울","연구"]
cnn_result = str(categories[result])       # 수신된 메시지 앞에 「echo:」 라는 메시지를 붙힌다.
print(cnn_result)

./KWGuideImg/dataset/test/test_img2.jpg
saebit
